In [2]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

url = "https://www.victoriassecret.com/us/pink/tops-and-bottoms/ultimate?scroll=true"

driver.get(url)
sleep(2)

soup = BeautifulSoup(driver.page_source,'html.parser')

links = []

for link in soup.select('li.fabric-grid-component-item > article > a'):
    #avoid duplicate link from image and title
    if link['href'] not in links:
        links.append(link['href'])
    else:
        continue

driver.quit()

len(links)

59

59 items

In [80]:
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output
import re

driver = webdriver.Firefox(options=options)

sleepTimes = [2.1, 2.8, 3.2]

other_links = []
info_text = []
for link in links:
    clear_output(wait=True)
    driver.get("https://www.victoriassecret.com" + link)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fabric-collapsible-info-component-body-fabrication-info-collapsible-info")))
    except:
        print("error")
    soup = BeautifulSoup(driver.page_source,'html.parser')

    # found some formatted as li, not p, so add to a separate list
    try:
        p = soup.select_one('#fabric-collapsible-info-component-body-fabrication-info-collapsible-info > span:nth-child(1) > p:nth-child(1)').text
    except:
        other_links.append(link)
        continue
    info = re.split(r"\s(?=\w+:)", p)
    info_text.extend(info)

    print("Current progress:", round( ( (links.index(link) + 1) / len(links) )*100, 2 ), "%")
    sleep(random.choice(sleepTimes))
    
driver.quit()
print(len(info_text))

Current progress: 100.0 %
102


In [89]:
driver = webdriver.Firefox(options=options)

sleepTimes = [2.1, 2.8, 3.2]

info_text2 = []
for link in other_links:
    clear_output(wait=True)
    driver.get("https://www.victoriassecret.com" + link)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fabric-collapsible-info-component-body-fabrication-info-collapsible-info")))
    except:
        print("error")
    soup = BeautifulSoup(driver.page_source,'html.parser')

    lis = soup.select('#fabric-collapsible-info-component-body-fabrication-info-collapsible-info > span > ul > li')
    for li in lis:
        info_text2.append(li.text)

    print("Current progress:", round( ( (other_links.index(link) + 1) / len(other_links) )*100, 2 ), "%")
    sleep(random.choice(sleepTimes))
    
driver.quit()
print(len(info_text2))

Current progress: 100.0 %
102


In [91]:
info_text.extend(info_text2)

In [96]:
import pandas as pd

materials_pink = pd.Series(info_text)
materials_pink.to_csv("materials_pink.csv")